In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_450x32x32_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_450x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train), to_tf(valid)
#train, valid = to_tf(train).cache(f'{tmp}/ours_32'), to_tf(valid).cache(f'{tmp}/ours_32')

In [33]:
m4 = M_1()
m4.compile(optimizer='Nadam', loss='mae')
m4.build(input_shape=(None, 450, 32, 32, 3))
print(f'Flops per frame:{get_flops(m4, input_sig=[tf.TensorSpec([1, 450, 32, 32, 3])])/450:.0f}')
m4.summary()

Flops per frame:649357
Model: "m_1_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_22 (Sequential)   (None, 450, 64)           196672    
_________________________________________________________________
spectral_transform_6 (Spectr multiple                  82560     
_________________________________________________________________
conv1d_20 (Conv1D)           multiple                  41024     
_________________________________________________________________
sequential_24 (Sequential)   (None, 450, 64)           256       
_________________________________________________________________
spectral_transform_7 (Spectr multiple                  49792     
_________________________________________________________________
conv1d_22 (Conv1D)           multiple                  10272     
_________________________________________________________________
sequential_26 (Sequential)   (None, 45

In [ ]:
m4.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/m4.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
m4.load_weights('../weights/m4.h5')

In [35]:
m4.load_weights('../weights/m4.h5')
#m4.cross()

In [39]:
eval_on_dataset(test_set_CCNU, m4, 450, (32, 32), step=1, batch=32, save='../results/ours_32_CCNU_CCNU.h5')
get_metrics('../results/ours_32_CCNU_CCNU.h5',)

100%|██████████| 179/179 [01:53<00:00,  1.58it/s]


{'Sliding window': {'MAE': 1.558, 'RMSE': 5.309, 'R': 0.86769},
 'Whole video': {'MAE': 0.83, 'RMSE': 2.878, 'R': 0.953}}

In [38]:
eval_on_dataset(test_set_CCNU_rPPG, m4, 450, (32, 32), step=1, batch=32, save='../results/ours_32_CCNU_CCNU_rPPG.h5')
get_metrics('../results/ours_32_CCNU_CCNU_rPPG.h5')

100%|██████████| 54/54 [00:24<00:00,  2.19it/s]


{'Sliding window': {'MAE': 1.523, 'RMSE': 5.189, 'R': 0.87072},
 'Whole video': {'MAE': 0.784, 'RMSE': 2.126, 'R': 0.97533}}

In [37]:
eval_on_dataset(test_set_PURE, m4, 450, (32, 32), step=1, batch=32, save='../results/ours_32_CCNU_PURE.h5')
get_metrics('../results/ours_32_CCNU_PURE.h5')

100%|██████████| 59/59 [00:14<00:00,  4.08it/s]


{'Sliding window': {'MAE': 0.425, 'RMSE': 1.29, 'R': 0.99847},
 'Whole video': {'MAE': 0.438, 'RMSE': 0.769, 'R': 0.99959}}

In [36]:
eval_on_dataset(test_set_UBFC_rPPG2, m4, 450, (32, 32), step=1, batch=32, save='../results/ours_32_CCNU_UBFC.h5')
get_metrics('../results/ours_32_CCNU_UBFC.h5')

100%|██████████| 42/42 [00:09<00:00,  4.40it/s]


{'Sliding window': {'MAE': 1.603, 'RMSE': 3.99, 'R': 0.97338},
 'Whole video': {'MAE': 0.827, 'RMSE': 1.473, 'R': 0.99684}}